In [9]:
import pandas as pd
data = pd.read_csv("carsdata_features.csv", engine='c')

In [10]:
data.head()

,NO_ABS,NO_accoudoir_central,NO_airbag_conducteur,NO_airbag_lateraux,NO_airbag_passager,NO_airbag_tete_rideau,alesage_mm,course_piston_mm,NO_allumage_auto_feux,NO_antipatinage,...,vitesse_max,NO_vitres_electriques,NO_vitres_teintees,voie_arriere,voie_avant,NO_volant_cuir,NO_volant_reglable,volume_coffre_min,volume_coffre_max,zero_100
0,0,1,0,0,0,0,86.0,86.00,1,0,...,220.0,0,0,1479.0,1462.0,0,0,150.0,1700.0,8.2
1,0,1,0,0,0,1,72.2,73.10,1,0,...,175.0,0,0,1476.0,1493.0,1,0,320.0,1200.0,11.1
2,0,0,0,0,0,1,89.9,100.00,1,0,...,176.0,0,0,1530.0,1530.0,0,0,410.0,1841.0,11.1
3,0,1,0,0,0,1,NaN,NaN,0,0,...,135.0,0,0,1510.0,1511.0,0,0,338.0,1225.0,13.5
4,0,0,0,0,0,0,77.0,85.44,0,0,...,180.0,0,0,1551.0,1547.0,0,0,440.0,1486.0,11.5


In [21]:
import numpy as np
cat_var = data.select_dtypes(include=['int64'])
cont_var = data.select_dtypes(include=['float64'])

In [22]:
cat_var.head()

,NO_ABS,NO_accoudoir_central,NO_airbag_conducteur,NO_airbag_lateraux,NO_airbag_passager,NO_airbag_tete_rideau,NO_allumage_auto_feux,NO_antipatinage,NO_appuis_tete_arriere,NO_assistant_freinage,...,suv_crossover,electromagnetic_synchronous_motor,valvetrain_ohc,valvetrain_ohv,permanent_magnet_synchronous_motor,NO_verre_intim_arriere,NO_vitres_electriques,NO_vitres_teintees,NO_volant_cuir,NO_volant_reglable
0,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,1,1,0,0,1,...,0,0,0,0,0,1,0,0,1,0
2,0,0,0,0,0,1,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
cont_var.head()

,alesage_mm,course_piston_mm,capacite_charge_max_kg,capacite_moteur_cc,conso_extra_urbaine_l_par_100km,conso_extra_urbaine_km_par_l,conso_mixte_l_par_100km,conso_mixte_km_par_l,conso_urbaine_l_par_100km,conso_urbaine_km_par_l,...,vitesse_1ere,vitesse_2eme,vitesse_3eme,vitesse_4eme,vitesse_max,voie_arriere,voie_avant,volume_coffre_min,volume_coffre_max,zero_100
0,86.0,86.00,625.0,1998.0,7.9,12.7,10.1,9.9,13.8,7.2,...,3.58,2.02,1.35,0.98,220.0,1479.0,1462.0,150.0,1700.0,8.2
1,72.2,73.10,563.0,898.0,4.3,23.3,5.2,19.2,6.7,14.9,...,3.73,1.96,1.23,0.90,175.0,1476.0,1493.0,320.0,1200.0,11.1
2,89.9,100.00,535.0,2488.0,7.9,12.7,9.6,10.4,12.7,7.9,...,2.79,1.55,1.00,0.69,176.0,1530.0,1530.0,410.0,1841.0,11.1
3,NaN,NaN,540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,135.0,1510.0,1511.0,338.0,1225.0,13.5
4,77.0,85.44,583.0,1591.0,5.2,19.2,6.5,15.4,8.6,11.6,...,4.40,2.73,1.83,1.39,180.0,1551.0,1547.0,440.0,1486.0,11.5
